In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
desired_directory = '/content/drive/MyDrive'

os.chdir(desired_directory)

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: /content/drive/MyDrive


In [ ]:
cd NLP_ass_2_3

/content/drive/MyDrive/NLP_ass_2_3


In [ ]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 6.1 MB/s eta 0:00:00


In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import re
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
column_names = ["comment", "label"]

In [ ]:
train = pd.read_csv('NLP_ass_train.tsv', sep='\t', header=None, names=column_names)
test = pd.read_csv('NLP_ass_test.tsv', sep='\t', header=None, names=column_names)
valid = pd.read_csv('NLP_ass_valid.tsv', sep='\t', header=None, names=column_names)

In [ ]:
train.iloc[:,0].values

array(['we cannot continue calling ourselves feminists if the rights of all womxn arent addressed yes to a sexual offences public list but will a trans lesbian bisexual and queer womxn be able to enter their information on the reporting sheet gender forum',
       'nawt yall niggers ignoring me',
       '<user> i am bit confused coz chinese ppl can not access twitter thn how this ching chong using it i think he pakistani 🤔 🤔 🤔',
       ...,
       'was macht der moslem wenn der zion gegen seinen propheten hetzt was machst du wenn die roten ratten gegen deinen toten opa hetzen',
       'it is awful look at world demographics asians are fucking everywhere another betrayal white genocide is real',
       'the jewish globalist elite have only imported a few million muslims to multiculturalize weaken america right now most all violence against lgbt is by black street criminals who are sometimes lgbt themselves and random as muslim grow i think we will start seeing organized attacks'],
     

In [ ]:
train

,comment,label
0,we cannot continue calling ourselves feminists...,normal
1,nawt yall niggers ignoring me,normal
2,<user> i am bit confused coz chinese ppl can n...,hatespeech
3,and this is why i end up with nigger trainee d...,hatespeech
4,nogs jews and dykes how enriching,offensive
...,...,...
15378,this vile racist ghetto ass dumber than a reta...,offensive
15379,when i first got on here and said i hate trump...,offensive
15380,was macht der moslem wenn der zion gegen seine...,normal
15381,it is awful look at world demographics asians ...,hatespeech


In [ ]:
train.head()

,comment,label
0,we cannot continue calling ourselves feminists...,normal
1,nawt yall niggers ignoring me,normal
2,<user> i am bit confused coz chinese ppl can n...,hatespeech
3,and this is why i end up with nigger trainee d...,hatespeech
4,nogs jews and dykes how enriching,offensive


In [ ]:

test.head()

,comment,label
0,i dont think im getting my baby them white 9 h...,normal
1,laura loomer raped me while screaming at me in...,hatespeech
2,<user> what did the old lady do was she a nazi...,normal
3,as much as i appreciate a format of open discu...,normal
4,sex be so good a bitch be slow stroking and cr...,offensive


In [ ]:
train.isnull().sum()

comment    0
label      0
dtype: int64

In [ ]:
# Map sentiment labels to numerical values
label_mapping = {'hatespeech': 2, 'offensive': 1, 'normal': 0}
train['label'] = train['label'].map(label_mapping)

test['label'] = test['label'].map(label_mapping)

valid['label'] = valid['label'].map(label_mapping)

In [ ]:
# Convert text to lowercase and remove special characters
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [ ]:
train['comment'] = train['comment'].apply(clean_text)
valid['comment'] = valid['comment'].apply(clean_text)
test['comment'] = test['comment'].apply(clean_text)

In [ ]:
# Split data into input text and sentiment label
x_train = train['comment'].values
y_train = np.eye(3)[train['label'].values]

# test data into input text
x_test = test['comment'].values
y_test = np.eye(3)[test['label'].values] # not doing the OHE

x_valid = valid['comment'].values
y_valid = np.eye(3)[valid['label'].values]

## Generic function

In [ ]:
def main(batch_size = 32,
          embedding_dim = 100,
          lstm_units = 128,
          dense_units = 64,
          dropout_prob = 0.5,
          bidirectional = False,
          max_words = 25000):

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(x_train)

    # Tokenize and pad the train and test data
    train_sequences = tokenizer.texts_to_sequences(x_train)
    valid_sequences = tokenizer.texts_to_sequences(x_valid)
    test_sequences = tokenizer.texts_to_sequences(x_test)

    vocab_size = len(tokenizer.word_index) + 1

    max_sequence_length = 50
    train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
    valid_padded = pad_sequences(valid_sequences, maxlen=max_sequence_length, padding='post')
    test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')


    # Convert to TensorFlow tensors
    x_train_tensor = tf.constant(train_padded, dtype=tf.int64)
    y_train_tensor = tf.constant(y_train, dtype=tf.int64)

    x_valid_tensor = tf.constant(valid_padded, dtype=tf.int64)
    y_valid_tensor = tf.constant(y_valid, dtype=tf.int64)

    x_test_tensor = tf.constant(test_padded, dtype=tf.int64)
    y_test_tensor = tf.constant(y_test, dtype=tf.int64)

    # Create a tf.data.Dataset from tensors
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train_tensor, y_train_tensor))
    valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid_tensor, y_valid_tensor))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test_tensor, y_test_tensor))



    num_classes = 3
    train = train_dataset.shuffle(buffer_size=len(train_dataset)).batch(batch_size)
    eval = valid_dataset.shuffle(buffer_size=len(valid_dataset)).batch(batch_size)

    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
    if bidirectional:
      model.add(Bidirectional(LSTM(units=lstm_units)))
    else:
      model.add(LSTM(units=lstm_units))
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_prob))
    model.add(Dense(num_classes))

    # Compile the model for multi-class classification
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy',tfa.metrics.F1Score(num_classes=3,
                                                  average='macro',
                                                  threshold=0.5)],run_eagerly=True)


    checkpoint_filepath = '/content/drive/MyDrive/NLP_ass2_3/Train/model3'
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)

    model = model.fit(
        train,
        validation_data=eval,
        epochs=20,
        callbacks=[model_checkpoint])

    model_best3 = load_model('/content/drive/MyDrive/NLP_ass2_3/Train/model3')

    test_loss, test_accuracy = model_best3.evaluate(x_test_tensor, y_test_tensor)

    print(f'Test Accuracy: {test_accuracy}')

## Lets experiment! Lets see if we can beat 62% test acc

In [ ]:
main(batch_size = 64,embedding_dim = 100,lstm_units = 64,dense_units = 64,dropout_prob = 0.5,bidirectional = True,max_words = 20000)

Epoch 1/20


241/241 [==============================] - 66s 248ms/step - loss: 0.9336 - accuracy: 0.5448 - f1_score: 0.3955 - val_loss: 0.8595 - val_accuracy: 0.5942 - val_f1_score: 0.4558
Epoch 2/20
241/241 [==============================] - 48s 201ms/step - loss: 0.6930 - accuracy: 0.7030 - f1_score: 0.6546 - val_loss: 0.8336 - val_accuracy: 0.6296 - val_f1_score: 0.6157
Epoch 3/20
241/241 [==============================] - 23s 97ms/step - loss: 0.5103 - accuracy: 0.7980 - f1_score: 0.7515 - val_loss: 0.9168 - val_accuracy: 0.6113 - val_f1_score: 0.6151
Epoch 4/20
241/241 [==============================] - 21s 86ms/step - loss: 0.3676 - accuracy: 0.8648 - f1_score: 0.8234 - val_loss: 1.0807 - val_accuracy: 0.6129 - val_f1_score: 0.6129
Epoch 5/20
241/241 [==============================] - 19s 78ms/step - loss: 0.2673 - accuracy: 0.9054 - f1_score: 0.8740 - val_loss: 1.2975 - val_accuracy: 0.5983 - val_f1_score: 0.5985
Epoch 6/20
241/241 [==============================] - 19s 80ms/step - loss: 0.1

ValueError: ignored

In [19]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(x_train)

# Tokenize and pad the train and test data
train_sequences = tokenizer.texts_to_sequences(x_train)
valid_sequences = tokenizer.texts_to_sequences(x_valid)
test_sequences = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

max_sequence_length = 50
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
valid_padded = pad_sequences(valid_sequences, maxlen=max_sequence_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')


# Convert to TensorFlow tensors
x_train_tensor = tf.constant(train_padded, dtype=tf.int64)
y_train_tensor = tf.constant(y_train, dtype=tf.int64)

x_valid_tensor = tf.constant(valid_padded, dtype=tf.int64)
y_valid_tensor = tf.constant(y_valid, dtype=tf.int64)

x_test_tensor = tf.constant(test_padded, dtype=tf.int64)
y_test_tensor = tf.constant(y_test, dtype=tf.int64)

In [20]:
model_best3 = load_model('/content/drive/MyDrive/NLP_ass2_3/Train/model3')

test_loss, test_accuracy, macroscore = model_best3.evaluate(x_test_tensor, y_test_tensor)
print(f'Test Accuracy: {test_accuracy}')
print(f'macro f1 score: {macroscore}')

61/61 [==============================] - 1s 6ms/step - loss: 0.8125 - accuracy: 0.6388 - f1_score: 0.6248
Test Accuracy: 0.6387733817100525
macro f1 score: 0.6247813105583191
